# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.116291e+02     1.598696e+00
 * time: 0.3358879089355469
     1     1.095175e+01     9.139767e-01
 * time: 1.9999639987945557
     2    -1.145615e+01     9.405409e-01
 * time: 2.646951913833618
     3    -3.394798e+01     7.202931e-01
 * time: 3.5712409019470215
     4    -4.746869e+01     5.526928e-01
 * time: 4.537462949752808
     5    -5.676666e+01     1.901569e-01
 * time: 5.478619813919067
     6    -5.972927e+01     1.070267e-01
 * time: 6.104112863540649
     7    -6.081086e+01     5.556482e-02
 * time: 6.764304876327515
     8    -6.119792e+01     6.650002e-02
 * time: 7.399596929550171
     9    -6.151828e+01     3.724827e-02
 * time: 8.003243923187256
    10    -6.173411e+01     3.404004e-02
 * time: 8.611342906951904
    11    -6.192297e+01     2.412175e-02
 * time: 9.233797788619995
    12    -6.200258e+01     1.989454e-02
 * time: 9.8659188747406
    13    -6.208473e+01     1.575111e-02
 * time: 10.485962867736816
    1

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671073
    AtomicLocal         -18.8557659
    AtomicNonlocal      14.8522622
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485371 
    Xc                  -19.3336820

    total               -62.261666458173
